In [ ]:
%matplotlib inline
import cv2
import imgaug
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

from imgaug import augmenters as ima

# Local imports
import adjust_path  # Before doing any local imports
from icc.data_loader import DataLoader
from icc.models.alexnet_model import AlexNet
from icc.models.spencer.alexnet.preprocessing import *

Ref:
https://github.com/aleju/imgaug

In [ ]:
# Load data
X, y = DataLoader.load_train()

In [ ]:
def plt_imgs(im, labels, c='gray'):
    # plt_imgs(timgs, y_train[:10])
    ids = ['ship', 'iceberg']
    
    for i in range(10):
        print('{}: {}'.format(i, ids[labels[i]]))
        if c=='gray':
            plt.imshow(np.squeeze(im[i]))
        plt.pause(1)

In [ ]:
def get_bands(X):
    band_1 = np.asarray(X['band_1'].tolist()).reshape(-1,75,75)
    band_1 = band_1[:,:,:,np.newaxis]
    band_2 = np.asarray(X['band_2'].tolist()).reshape(-1,75,75)
    band_2 = band_2[:,:,:,np.newaxis]
    return [band_1, band_2]

def emboss_and_shear(imgs):
    seq = ima.Sequential([ima.Emboss(alpha=1, strength=2.0), ima.Affine(scale=1.0, shear=45)])
    return seq.augment_images(imgs)
    
def affine_and_translate(imgs):
    seq = ima.Sequential([ima.Affine(scale=1.0, translate_percent=0.2)])
    return seq.augment_images(imgs)

def constrast_norm(imgs):
    seq = ima.Sequential([ima.ContrastNormalization((0.4, 0.6))])
    return seq.augment_images(imgs)

def crop_and_resize(imgs):
    cropped = np.squeeze(imgs)[:, 10:10+50, 10:10+50]
    m = len(cropped)
    h, w = 75, 75

    resized = np.zeros((m, h, w))
    for i,img in enumerate(cropped):
        resized[i,:,:] = cv2.resize(img, (h, w), interpolation= cv2.INTER_AREA)
    return resized[:,:,:,np.newaxis]

def flipLR_and_rotate(imgs):
    seq = ima.Sequential([ima.Fliplr((0.6)), ima.Affine(scale=1.0, rotate=90)])
    return seq.augment_images(imgs)

In [ ]:
def data_augmentation_edit():
    band_1, band_2 = get_bands(X)
    
    b1_augmented = np.concatenate([band_1, 
                                   emboss_and_shear(band_1), 
                                   affine_and_translate(band_1),
                                   crop_and_resize(band_1),
                                   flipLR_and_rotate(band_1),
                                   constrast_norm(band_1)])

    b2_augmented = np.concatenate([band_2, 
                                   emboss_and_shear(band_2), 
                                   affine_and_translate(band_2),
                                   crop_and_resize(band_2),
                                   flipLR_and_rotate(band_2),
                                   constrast_norm(band_2)])
    
    return pd.DataFrame({'band_1': np.squeeze(b1_augmented).reshape(-1,75*75).tolist(), 
                         'band_2': np.squeeze(b2_augmented).reshape(-1,75*75).tolist()})

In [ ]:
data = data_augmentation_edit()

data['band_1'] = data['band_1'].map(np.array)
data['band_2'] = data['band_2'].map(np.array)
data['is_iceberg'] = np.tile(y, 6)

In [ ]:
data.to_json('../data/train_augmented.json')